# Dimensionality Reduction

#imports 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


In [ ]:
#read in demographics df 
mixed_demographic = pd.read_csv('../data:/training_data/mixed_demographics.csv')
mixed_demographics.head()

In [ ]:
#read in cloud data